In [105]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [115]:
class Node:
    def __init__(self, feature=None, f_value=None, childs=None, label=None, most_common=None):
        self.feature = feature
        self.f_value = f_value
        self.childs = childs
        self.label = label
        self.most_common = most_common
        
    def is_leaf_node(self):
        return self.label is not None

In [116]:
class DecisionTree:
    def __init__(self, min_samples_split=1):
        self.min_samples_split=min_samples_split
        self.root=None

    def fit(self, X, y):
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, f_value=None):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(f_value=f_value, label=leaf_value, most_common=leaf_value)

        # find the best split
        best_feature = self._best_split(X, y)

        # create child nodes
        childs = {}
        best_column = X[:, best_feature]
        for v in np.unique(best_column):
            index = np.argwhere(best_column == v).flatten()
            child_X = X[index]
            child_y = y[index]
            childs[v] = self._grow_tree(child_X, child_y, v)
        
        return Node(feature=best_feature, f_value=f_value, childs=childs, most_common=self._most_common_label(y))


    def _best_split(self, X, y):
        best_gain = -1
        split_idx = None

        for feat_idx in range(X.shape[1]):
            X_column = X[:, feat_idx]
            
            gain = self._information_gain(y, X_column)
            
            if gain > best_gain:
                best_gain = gain
                split_idx = feat_idx

        return split_idx


    def _information_gain(self, y, X_column):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        childs_idx = self._split(X_column)
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        information_gain = parent_entropy
#         print('an')
        for ch_id in childs_idx:
            n_ch = len(ch_id)
#             print(n_ch,'/', n)
            information_gain -= (n_ch / n) * self._entropy(y[ch_id])
            
        return information_gain

    def _split(self, X_column):
        splits = []
        for v in np.unique(X_column):
            splits.append(np.argwhere(X_column == v).flatten())
        return splits

    def _entropy(self, y):
#         hist = np.bincount(y)
#         ps = hist / len(y)
#         return -np.sum([p * np.log2(p) for p in ps if p>0])
        count = Counter(y)
        return -np.sum([count[i]/len(y) * np.log2(count[i]/len(y)) for i in count])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.label
        if x[node.feature] in node.childs:
            return self._traverse_tree(x, node.childs[x[node.feature]])
        else:
            return node.most_common
        

# Test Bed

In [4]:
import pandas as pd

data = {
    'alt': ['Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes'],
    'bar': ['No', 'No', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes'],
    'fri': ['No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes'],
    'hun': ['Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes'],
    'pat': ['Some', 'Full', 'Some', 'Full', 'Full', 'Some', 'None', 'Some', 'Full', 'Full', 'None', 'Full'],
    'price': ['$$$', '$', '$', '$', '$$$', '$$', '$', '$$', '$', '$$$', '$', '$'],
    'rain': ['No', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No'],
    'res': ['Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No'],
    'type': ['French', 'Thai', 'Burger', 'Thai', 'French', 'Italian', 'Burger', 'Thai', 'Burger', 'Italian', 'Thai', 'Burger'],
    'est': ['0-10', '30-60', '0-10', '10-30', '>60', '0-10', '0-10', '0-10', '>60', '10-30', '0-10', '30-60'],
    'will_wait': ['Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes']
}
df = pd.DataFrame(data)
df

,alt,bar,fri,hun,pat,price,rain,res,type,est,will_wait
0,Yes,No,No,Yes,Some,$$$,No,Yes,French,0-10,Yes
1,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
2,No,Yes,No,No,Some,$,No,No,Burger,0-10,Yes
3,Yes,No,Yes,Yes,Full,$,Yes,No,Thai,10-30,Yes
4,Yes,No,Yes,No,Full,$$$,No,Yes,French,>60,No
5,No,Yes,No,Yes,Some,$$,Yes,Yes,Italian,0-10,Yes
6,No,Yes,No,No,None,$,Yes,No,Burger,0-10,No
7,No,No,No,Yes,Some,$$,Yes,Yes,Thai,0-10,Yes
8,No,Yes,Yes,No,Full,$,Yes,No,Burger,>60,No
9,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,Italian,10-30,No


In [85]:
def traverse(root, depth=0, his=[]):
    if root.feature is not None:
        if depth < len(his):
            his[depth].append((root.f_value, list(df.columns)[root.feature]))
        else:
            his.append([(root.f_value, list(df.columns)[root.feature])])
    else:
        if depth < len(his):
            his[depth].append((root.f_value, root.label))
        else:
            his.append([(root.f_value, root.label)])
    
    if root.childs is not None:
        for child_key in root.childs:
            his = traverse(root.childs[child_key], depth+1, his=his)
    
    return his

In [117]:
tree = DecisionTree(min_samples_split=1)
tree.fit(df.to_numpy()[:, :-1], df.to_numpy()[:, -1])

In [119]:
y_pred = tree.predict(df.to_numpy()[:, :-1])
# print(accuracy_score(y_pred, df.to_numpy()[:, -1]))
print(y_pred)

[0.1189  0.08902 0.08758 0.173   0.07678 0.1244  0.08368 0.1151  0.1072
 0.2075  0.08452 0.1048  0.1023  0.06287 0.1431  0.1341  0.08216 0.1142
 0.07615 0.07259 0.08183 0.07773 0.09946 0.07526 0.09564 0.1059  0.1275
 0.07421 0.09876 0.07919 0.09782 0.1402  0.08482 0.1123  0.1233  0.08633
 0.1014  0.06169 0.05504 0.1071  0.07146 0.09606 0.1038  0.1027  0.09618
 0.09185 0.07409 0.1179  0.08301 0.06917 0.06563 0.08025 0.07408 0.07987
 0.07873 0.07036 0.08294 0.1094  0.06289 0.09026 0.0802  0.07712 0.1132
 0.0849  0.1031  0.08911 0.09211 0.06641 0.1175  0.0641  0.06589 0.1084
 0.1339  0.103   0.07609 0.06387 0.07191 0.1108  0.09964 0.07918 0.08851
 0.1016  0.1051  0.09203 0.07924 0.08579 0.06846 0.09288 0.09261 0.08473
 0.07246 0.06828 0.06206 0.06603 0.08234 0.08368 0.07376 0.08988 0.08756
 0.09353 0.07397 0.09382 0.06878 0.0849  0.07552 0.1405  0.09097 0.07185
 0.09789 0.08832 0.08468 0.08486 0.1082  0.09026 0.1017  0.08541 0.07722
 0.1065  0.1252  0.06111 0.08523 0.08456 0.08009 0.08006

In [88]:
traverse(tree.root)

[[(None, 'pat')],
 [('Full', 'type'), ('None', 'No'), ('Some', 'Yes')],
 [('Burger', 'alt'), ('French', 'No'), ('Italian', 'No'), ('Thai', 'fri')],
 [('No', 'No'), ('Yes', 'Yes'), ('No', 'No'), ('Yes', 'Yes')]]

# Cancer dataset

In [95]:
df = pd.read_csv('part2.csv').iloc[:, 1:-1]
df

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [108]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

for c in X.columns:
    X[c] = pd.cut(X[c], 10)

x_train, x_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), test_size=0.2)
    
my_tree = DecisionTree()
my_tree.fit(x_train, y_train)


In [110]:
y_pred = my_tree.predict(x_test)
y_pred

KeyError: Interval(11.982, 15.772, closed='right')